In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from PIL import Image
from fastai.vision.all import *

In [ ]:
parquet_path=Path("/home/george/codes/lepinet/data/mini/0013397-241007104925546_processing_metadata_postprocessed.parquet")
images_path=Path("/home/george/codes/lepinet/data/mini/images")
root_path=Path("/home/george/codes/lepinet/data/mini")
export_path=Path("/home/george/codes/lepinet/data/mini/models")

In [ ]:
df=pd.read_parquet(parquet_path)

## Result analysis

* F1 macro on the validation set
* F1 macro on the test set

In [ ]:
model_path = export_path / "00_lepi_mini_model1"

learn = load_learner(model_path)
learn.model.eval().to("cuda")

In [ ]:
def prepare_df(df, remove_in=[], keep_in=[]):
    # Filter out 'test_ood' rows and 'test_in' rows
    if len(remove_in)>0:
        df = df[~df['set'].isin(remove_in)]
    if len(keep_in)>0:
        df = df[df['set'].isin(keep_in)]
    def generate_image_path(row):
        return Path(str(row['speciesKey'])) / row['filename']

    # Apply the function to create the image paths
    df['image_path'] = df.apply(generate_image_path, axis=1)
    # Add a column to specify whether the row is for training or validation
    df['is_valid'] = df['set'] == '0'
    # Define the hierarchical levels
    hierarchy_levels = ["familyKey", "genusKey", "speciesKey"]

    # Create a function to extract the labels at different hierarchy levels
    def get_hierarchy_labels(row):
        return ' '.join(map(str, [row[level] for level in hierarchy_levels]))

    # Add a column with hierarchy labels
    df['hierarchy_labels'] = df.apply(get_hierarchy_labels, axis=1)
    # Keep only the columns needed for ImageDataLoaders
    df = df[['image_path', 'hierarchy_labels', 'is_valid']]
    return df

df_val = prepare_df(pd.read_parquet(parquet_path), keep_in=["0"])
df_train = prepare_df(pd.read_parquet(parquet_path), remove_in=["test_ood", "0"])

In [ ]:
df_val.head()

In [ ]:
# test on one image
learn.predict(images_path/df["image_path"].iloc[0])

In [ ]:
interp = Interpretation.from_learner(learn)
interp.plot_top_losses(9)